In [85]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

 # 读取数据
data = pd.read_csv(r"D:\DL_Homework\Kaggle2_Titanic\统计建模\数据\data-1.csv", encoding='gb18030')
 # 将 'date' 列转换为日期时间格式
data['date'] = pd.to_datetime(data['date'])
 # 添加年、月、日列
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data = data.set_index('date')
cols_to_convert = ['AQI指数', 'PM2.5', 'PM10', 'O3', 'no2', 'so2', 'co', 'T', 'Po', 'U', 'Ff', 'VV', 'RRR', 'year', 'month', 'day']
data[cols_to_convert] = data[cols_to_convert].astype(float)

# y = data['AQI指数']
# y = stats.boxcox(y)[0]  #对数据进行BOX-COX变换
# x = data.drop(['AQI指数'], axis=1)
scaler = StandardScaler()
data = scaler.fit_transform(data)


In [86]:
def split_data(data, test_size=0.2, analysis='univariate'):

    if analysis == 'univariate':
        position = int(round(len(data) * (1-test_size)))
        train = np.array(data[:position]).reshape(-1,1)
        test = np.array(data[position:]).reshape(-1,1)

    elif analysis == 'multivariate':
        position = int(round(len(data) * (1-test_size)))
        train = data[:position]
        test = data[position:]

    else:
        print("Please specify if analysis is univariate or multivariate")

    return train, test

def scale(data, train_split):

    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)

    return (data - data_mean) / data_std

def inv_scale(data, scaled_data, train_split):

    data_mean = data[:train_split].mean(axis=0)
    data_std = data[:train_split].std(axis=0)

    return scaled_data * data_std + data_mean
test_size = 0.25
ddf = data
TRAIN_SPLIT = int(round(len(ddf) * (1-test_size)))
TRAIN_SPLIT

2173

In [89]:
multivar_data = ddf
multivar_data.shape

(2897, 16)

In [90]:
multivar_data_scaled = scale(multivar_data, TRAIN_SPLIT)
multivar_data_scaled.shape, multivar_data_scaled[0]

test_size=0.2
train, test = split_data(data=multivar_data_scaled, test_size=test_size, analysis='multivariate')


# print("Shape train:", train.shape, "\nShape test:", test.shape, '\n')
# print(train[:9])
# print(test[:9])

In [91]:
def create_samples_multivar(data, n_input, n_output, target_index=0):
    """
    Function to convert time series observations
    into input X and output y
    """
    X, y = [], []

    # Move column defined  y target_index to first position
    a = data[:, target_index].reshape(-1,1)
    b = np.delete(data, target_index, axis=1)
    data = np.concatenate((a, b), axis=1)

    for i in range(n_input, len(data)): #start, stop
        if i+n_output > len(data):
            break
        X.append(data[i-n_input : i, :])
        y.append(data[i : i+n_output, 0])

    return np.array(X), np.array(y)
n_input = 60 #n_input表示使用前多少个单位时间步长作为预测因变量
n_output = 60 #n_output表示对接下来多少个时刻进行预测，此处表示对接下来60分钟的用电量进行预测
target_index = 0 #target_index控制的是预测哪个特征变量，0表示对“Global_active_power”进行预测，1表示对“Global_reactive_power”进行预测，以此类推

X_train, y_train = create_samples_multivar(train, n_input=n_input, n_output=n_output, target_index=target_index)
X_test, y_test = create_samples_multivar(test, n_input=n_input, n_output=n_output, target_index=target_index)

#
# print("Shape X_train y_train:", X_train.shape, y_train.shape,
#       "\nShape X_test y_test:", X_test.shape, y_test.shape, '\n')
# for i in range(1):
#     print(X_train[i], y_train[i])

In [101]:
from keras.optimizers import Adam
window_size = 60
fea_num = 16


# model = keras.models.Sequential([
#     keras.layers.Input((window_size, fea_num)),
#     keras.layers.Reshape((window_size, fea_num, 1)),
#     keras.layers.Conv2D(filters=64, kernel_size=(2,1), strides=1, padding="same", activation="relu"),
#     keras.layers.MaxPooling2D(pool_size=(2,1), strides=1, padding="same"),
#     keras.layers.Conv2D(filters=64, kernel_size=(2,1), strides=1, padding="same", activation="relu"),
#     keras.layers.MaxPooling2D(pool_size=(2,1), strides=1, padding="same"),
#
#     keras.layers.Reshape((window_size, -1)),
#     keras.layers.LSTM(64, return_sequences=False, activation="tanh"),
#     keras.layers.Dense(32, activation="relu"),
#     keras.layers.Dense(10)
# ])

model = tf.keras.Sequential([
    # tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(60,16)),
    # tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(60)
])




model.compile(loss='mse', optimizer=Adam(0.001), metrics=['mse'])

In [102]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

epochs=200
verbose=1

# Initialize empty lists to store metrics
mse_list = []
rmse_list = []
mae_list = []
r2_list = []
mape_list = []

# Training loop
for epoch in range(epochs):
    # Perform training epoch
    history = model.fit(X_train, y_train, epochs=1, verbose=verbose)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    # Append metrics to the lists
    mse_list.append(mse)
    rmse_list.append(rmse)
    mae_list.append(mae)
    r2_list.append(r2)
    mape_list.append(mape)

    # Print metrics for the current epoch
    print(f"Epoch {epoch + 1} - MSE: {mse:.4f} - RMSE: {rmse:.4f} - MAE: {mae:.4f} - R2: {r2:.4f} - MAPE: {mape:.2f}%")

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({
    'Epoch': range(1, epochs + 1),
    'MSE': mse_list,
    'RMSE': rmse_list,
    'MAE': mae_list,
    'R2': r2_list,
    'MAPE': mape_list
})

# Print the metrics table
print(metrics_df)


15/15 [==============================] - 2s 33ms/step
Epoch 1 - MSE: 0.7242 - RMSE: 0.8510 - MAE: 0.6539 - R2: 0.3095 - MAPE: 150.29%
15/15 [==============================] - 1s 38ms/step
Epoch 2 - MSE: 0.6963 - RMSE: 0.8345 - MAE: 0.6262 - R2: 0.3340 - MAPE: 155.22%
15/15 [==============================] - 1s 37ms/step
Epoch 3 - MSE: 0.7621 - RMSE: 0.8730 - MAE: 0.6679 - R2: 0.2715 - MAPE: 176.78%
15/15 [==============================] - 1s 34ms/step
Epoch 4 - MSE: 0.8246 - RMSE: 0.9080 - MAE: 0.7110 - R2: 0.2090 - MAPE: 187.91%
15/15 [==============================] - 1s 34ms/step
Epoch 5 - MSE: 0.8458 - RMSE: 0.9197 - MAE: 0.7253 - R2: 0.1924 - MAPE: 187.81%
18/69 [======>.......................] - ETA: 4s - loss: 0.6013 - mse: 0.6013

KeyboardInterrupt: 